In [7]:
%pip install pandas
%pip install Faker

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import zipfile
import pandas as pd
from pathlib import Path
import urllib.request

SEED = 42

def load_kaggle_data():
    file_path = Path("./datasets/archive.zip")
    if not file_path.is_file():
        Path("datasets").mkdir(parents=True, exist_ok=True)
        url = "https://www.kaggle.com/api/v1/datasets/download/amirmahdiabbootalebi/salary-by-job-title-and-country"
        urllib.request.urlretrieve(url, file_path)
    zf = zipfile.ZipFile("./datasets/archive.zip")

    return pd.read_csv(zf.open('Salary.csv'))

kaggle_data = load_kaggle_data ()
print (*kaggle_data.columns, sep='\n')

Age
Gender
Education Level
Job Title
Years of Experience
Salary
Country
Race
Senior


In [9]:
from faker import Faker
import random

# Copy dataset and only take the columns we need
dataset_clean = kaggle_data[["Salary", "Country", "Job Title"]]

# Rename the columns that were copied
dataset_clean = dataset_clean.rename (columns={"Salary": "salary", "Country": "work_location", "Job Title": "job_role"})

# Create mock data for the remaininig columns
Faker.seed(SEED)
faker = Faker()

dataset_clean["name"] = [faker.name() for i in range (len (dataset_clean))]
dataset_clean["phone_number"] = [f"{random.randint(200, 999):03d}{random.randint(200, 999):03d}{random.randint(200, 999):04d}" for i in range (len (dataset_clean))]

print (*dataset_clean.columns, sep="\n")
print (dataset_clean.head ())

salary
work_location
job_role
name
phone_number
     salary work_location           job_role             name phone_number
0   90000.0            UK  Software Engineer     Allison Hill   3494000332
1   65000.0           USA       Data Analyst      Noah Rhodes   3885180614
2  150000.0        Canada            Manager  Angie Henderson   9797910708
3   60000.0           USA    Sales Associate    Daniel Wagner   7579040229
4  200000.0           USA           Director  Cristian Santos   2402910568


In [11]:
import subprocess

# Save the data to the disk
FILENAME = "./datasets/dataset_clean"
dataset_clean.to_parquet (FILENAME + ".parquet")
dataset_clean.to_json (FILENAME + ".json", orient='records', indent=4)

# Save the data to the database
try:
    MONGO_URI = "mongodb://localhost:27017/employeeDB"
    MONGO_COLLECTION = "employees"
    result = subprocess.run(["mongoimport", "--uri", MONGO_URI, "--collection", MONGO_COLLECTION, "--drop", "--file", FILENAME + ".json", "--jsonArray"], capture_output=True, text=True, check=True)

    print("Standard Output:")
    print(result.stdout)
    print("Standard Error:")
    print(result.stderr)
except subprocess.CalledProcessError as e:
    print(f"Error executing command: {e}")
    print(f"Return Code: {e.returncode}")
    print(f"Output: {e.output}")
    print(f"Error Output: {e.stderr}")

Standard Output:

Standard Error:
2025-07-10T09:34:43.302-0400	connected to: mongodb://localhost:27017/employeeDB
2025-07-10T09:34:43.315-0400	dropping: employeeDB.employees
2025-07-10T09:34:43.571-0400	6684 document(s) imported successfully. 0 document(s) failed to import.

